### Import Library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.api import add_constant
from statsmodels.regression.linear_model import OLS
from statsmodels.sandbox.regression.gmm import IV2SLS
from linearmodels.system import SUR
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
import scipy.stats as stats

### Read Dataset

In [2]:
data = pd.read_excel('econometrics_sur_eastJava.xlsx')
data.head()

,district_city,grdp,hdi,minimum_wage,regional_income,total_workforce,poverty,ou_rate
0,Bangkalan,1.20,66.82,2.152451,2220.215269,608798,196.66,6.18
1,Banyuwangi,5.03,73.79,2.528899,3236.145966,1095024,119.52,4.75
2,Blitar,4.45,72.84,2.215071,2445.113221,734660,101.94,4.91
3,Bojonegoro,2.47,71.80,2.279568,5767.294393,786549,153.25,4.63
4,Bondowoso,4.62,70.56,2.154504,1908.478394,468642,105.13,4.15


### Check Info

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   district_city    38 non-null     object 
 1   grdp             38 non-null     float64
 2   hdi              38 non-null     float64
 3   minimum_wage     38 non-null     float64
 4   regional_income  38 non-null     float64
 5   total_workforce  38 non-null     int64  
 6   poverty          38 non-null     float64
 7   ou_rate          38 non-null     float64
dtypes: float64(6), int64(1), object(1)
memory usage: 2.5+ KB


### Model 1 - GRDP

In [4]:
X1 = add_constant(data[['hdi','minimum_wage' ,'poverty', 'ou_rate', 'total_workforce']])
y1 = data['grdp']

### Model 2 - Total Workforce

In [5]:
X2 = add_constant(data[['grdp', 'hdi', 'minimum_wage', 'regional_income', 'poverty', 'ou_rate']])
y2 = data['total_workforce']

### Multicollinearity Test - VIF Model 1

In [6]:
vif = pd.DataFrame()
vif["variable"] = X1.columns
vif["VIF"] = [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]
vif

,variable,VIF
0,const,910.774670
1,hdi,3.927465
2,minimum_wage,1.970665
3,poverty,5.476245
4,ou_rate,2.129289
5,total_workforce,3.667442


### Multicollinearity Test - VIF Model 2

In [7]:
vif = pd.DataFrame()
vif["variable"] = X2.columns
vif["VIF"] = [variance_inflation_factor(X2.values, i) for i in range(X2.shape[1])]
vif

,variable,VIF
0,const,935.864023
1,grdp,1.252480
2,hdi,4.106407
3,minimum_wage,2.142916
4,regional_income,2.142958
5,poverty,3.707104
6,ou_rate,2.078266


### Normality Test

In [8]:
# perform the regression
model1 = sm.OLS(y1, X1).fit()
model2 = sm.OLS(y2, X2).fit()

# residuals from model1 and model2
residual1 = model1.resid
residual2 = model2.resid

# perform Kolmogorov-Smirnov test on residual model 1
ks_stat1, ks_p_value1 = stats.kstest(residual1, 'norm', args=(np.mean(residual1), np.std(residual1)))
print('Kolmogorov-Smirnov Test for residual Model 1 - GRDP, p-value:', ks_p_value1)

# perform Kolmogorov-Smirnov test on residual model 2
ks_stat2, ks_p_value2 = stats.kstest(residual2, 'norm', args=(np.mean(residual2), np.std(residual2)))
print('Kolmogorov-Smirnov Test for residual Model 2 - Total Workforce, p-value:', ks_p_value2)


Kolmogorov-Smirnov Test for residual Model 1 - GRDP, p-value: 0.2508158958406136
Kolmogorov-Smirnov Test for residual Model 2 - Total Workforce, p-value: 0.742277442379595


### Homoscedasticity Test

In [9]:
# Breusch-Pagan Test
bp_test1 = het_breuschpagan(residual1, X1)
bp_test2 = het_breuschpagan(residual2, X2)

print('Breusch-Pagan Test for Model 1 - GRDP:', bp_test1)
print('Breusch-Pagan Test for Model 2 - Total Workforce:', bp_test2)

Breusch-Pagan Test for Model 1 - GRDP: (8.05835076809901, 0.15304898662781297, 1.7224650691881498, 0.1578453063390746)
Breusch-Pagan Test for Model 2 - Total Workforce: (8.07257203530406, 0.23283457712783817, 1.3936476231192465, 0.24841916205716916)


### Autocorrelation Test

In [10]:
# Durbin-Watson Test for model1 and model2
dw1 = durbin_watson(residual1)
dw2 = durbin_watson(residual2)

print('Durbin-Watson Test for Model 1 - GRDP:', dw1)
print('Durbin-Watson Test for Model 2 - Total Workforce:', dw2)

Durbin-Watson Test for Model 1 - GRDP: 2.3339705228112266
Durbin-Watson Test for Model 2 - Total Workforce: 2.130209315741614


### Model SUR

In [11]:
equations = {
    'grdp': (y1, X1),
    'total_workforce': (y2, X2)
}

### SUR Estimate

In [12]:
sur_model = SUR(equations)
results = sur_model.fit()

### Display Result

In [13]:
results.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                           System GLS Estimation Summary                           
===================================================================================
Estimator:                        GLS   Overall R-squared:                   0.8821
No. Equations.:                     2   McElroy's R-squared:                 0.8304
No. Observations:                  38   Judge's (OLS) R-squared:             0.8821
Date:                Fri, Nov 29 2024   Berndt's R-squared:                  0.9267
Time:                        21:31:53   Dhrymes's R-squared:                 0.8821
                                        Cov. Estimator:                      robust
                                        Num. Constraints:                      None
                      Equation: grdp, Dependent Variable: grdp                     
===================================================================================
                 Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------
const               5.1933     5.2967     0.9805     0.3268     -5.1880      15.575
hdi                -0.0025     0.0814    -0.0308     0.9754     -0.1620      0.1570
minimum_wage        0.3681     0.1870     1.9689     0.0490      0.0017      0.7345
poverty            -0.0162     0.0043    -3.7686     0.0002     -0.0247     -0.0078
ou_rate            -0.2437     0.1930    -1.2627     0.2067     -0.6219      0.1346
total_workforce  2.602e-06  6.979e-07     3.7282     0.0002   1.234e-06    3.97e-06
           Equation: total_workforce, Dependent Variable: total_workforce          
===================================================================================
                 Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------
const           -5.607e+05  6.391e+05    -0.8774     0.3803  -1.813e+06   6.918e+05
grdp             9.908e+04  2.282e+04     4.3417     0.0000   5.435e+04   1.438e+05
hdi                -836.10     9393.0    -0.0890     0.9291  -1.925e+04   1.757e+04
minimum_wage    -3.657e+04   2.93e+04    -1.2480     0.2120  -9.399e+04   2.086e+04
regional_income     122.12     24.156     5.0555     0.0000      74.775      169.46
poverty             3385.1     722.01     4.6884     0.0000      1970.0      4800.2
ou_rate           4.31e+04  1.809e+04     2.3828     0.0172      7647.2   7.854e+04
===================================================================================

Covariance Estimator:
Heteroskedastic (Robust) Covariance (Debiased: False, GLS: True)
"""

### Correlation Between Two Equations

In [14]:
residuals = results.resids
residual_grdp = residuals['grdp']
residual_total_workforce = residuals['total_workforce']

corr_grdp_total_workforce = np.corrcoef(residual_grdp, residual_total_workforce)[0, 1]
print(f'Korelasi antara residual GRDP dan Total Workforce: {corr_grdp_total_workforce:.4f}')

Korelasi antara residual GRDP dan Total Workforce: -0.5523
